# Creating and modelling metallic supercells

In this section we will be concerned with modelling supercells of aluminium.
When dealing with periodic problems there is no unique definition of the
lattice: Clearly any duplication of the lattice along an axis is also a valid
repetitive unit to describe exactly the same system.
This is exactly what a **supercell** is: An $n$-fold repetition along one of the
axes of the original lattice.

The following code achieves this for aluminium:

In [1]:
using DFTK
using LinearAlgebra
using ASEconvert

function aluminium_setup(repeat=1; Ecut=7.0, kgrid=[2, 2, 2])
    a = 7.65339
    lattice = a * Matrix(I, 3, 3)
    Al = ElementPsp(:Al, psp=load_psp("hgh/lda/al-q3"))
    atoms     = [Al, Al, Al, Al]
    positions = [[0.0, 0.0, 0.0], [0.0, 0.5, 0.5], [0.5, 0.0, 0.5], [0.5, 0.5, 0.0]]
    unit_cell = periodic_system(lattice, atoms, positions)

    # Make supercell in ASE:
    # We convert our lattice to the conventions used in ASE, make the supercell
    # and then convert back ...
    supercell_ase = convert_ase(unit_cell) * pytuple((repeat, 1, 1))
    supercell     = pyconvert(AbstractSystem, supercell_ase)

    # Unfortunately right now the conversion to ASE drops the pseudopotential information,
    # so we need to reattach it:
    supercell = attach_psp(supercell, Al="hgh/lda/al-q3")

    # Construct an LDA model and discretise
    # Note: We disable symmetries explicitly here. Otherwise the problem sizes
    #       we are able to run on the CI are too simple to observe the numerical
    #       instabilities we want to trigger here.
    model = model_LDA(supercell; temperature=1e-3, symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid)
end;

As part of the code we are using a routine inside the ASE,
the [atomistic simulation environment](https://wiki.fysik.dtu.dk/ase/index.html)
for creating the supercell and make use of the two-way interoperability of
DFTK and ASE. For more details on this aspect see the documentation
on Input and output formats.

Write an example supercell structure to a file to plot it:

In [2]:
setup = aluminium_setup(5)
convert_ase(periodic_system(setup.model)).write("al_supercell.png")

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/XF0Hn/src/ASEconvert.jl:144
┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/XF0Hn/src/ASEconvert.jl:144


Python None

<img src="https://docs.dftk.org/stable/examples/al_supercell.png" width=500 height=500 />

As we will see in this notebook the modelling of a system generally becomes
harder if the system becomes larger.

- This sounds like a trivial statement as *per se* the cost per SCF step increases
  as the system (and thus $N$) gets larger.
- But there is more to it:
  If one is not careful also the *number of SCF iterations* increases
  as the system gets larger.
- The aim of a proper computational treatment of such supercells is therefore
  to ensure that the **number of SCF iterations remains constant** when the
  system size increases.

For achieving the latter DFTK by default employs the `LdosMixing`
preconditioner [^HL2021] during the SCF iterations. This mixing approach is
completely parameter free, but still automatically adapts to the treated
system in order to efficiently prevent charge sloshing. As a result,
modelling aluminium slabs indeed takes roughly the same number of SCF iterations
irrespective of the supercell size:

[^HL2021]:
   M. F. Herbst and A. Levitt.
   *Black-box inhomogeneous preconditioning for self-consistent field iterations in density functional theory.*
   J. Phys. Cond. Matt *33* 085503 (2021). [ArXiv:2009.01665](https://arxiv.org/abs/2009.01665)

In [3]:
self_consistent_field(aluminium_setup(1); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/XF0Hn/src/ASEconvert.jl:144
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -8.298394276509                   -0.85    4.9         
  2   -8.300171737251       -2.75       -1.26    1.0    133ms
  3   -8.300429637219       -3.59       -1.89    3.0    192ms
  4   -8.300462141739       -4.49       -2.76    2.1    175ms
  5   -8.300464132521       -5.70       -3.06    2.8    202ms
  6   -8.300464396865       -6.58       -3.25    9.2    455ms
  7   -8.300464516702       -6.92       -3.40    2.4    225ms
  8   -8.300464588440       -7.14       -3.55    1.4    194ms
  9   -8.300464627264       -7.41       -3.74    1.4    196ms
 10   -8.300464638453       -7.95       -3.91    1.6    197ms
 11   -8.300464643760       -8.28       -4.28    3.2    288ms


In [4]:
self_consistent_field(aluminium_setup(2); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/XF0Hn/src/ASEconvert.jl:144
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -16.67413258720                   -0.70    5.5         
  2   -16.67855912303       -2.35       -1.14    1.9    358ms
  3   -16.67920796735       -3.19       -1.86    3.2    400ms
  4   -16.67927884068       -4.15       -2.74    3.8    483ms
  5   -16.67928599440       -5.15       -3.17    5.9    562ms
  6   -16.67928620053       -6.69       -3.44    2.1    442ms
  7   -16.67928622108       -7.69       -3.92    1.9    353ms
  8   -16.67928622400       -8.53       -4.55    2.8    419ms


In [5]:
self_consistent_field(aluminium_setup(4); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/XF0Hn/src/ASEconvert.jl:144
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -33.32562740713                   -0.56    7.1         
  2   -33.33267850892       -2.15       -1.00    1.1    1.26s
  3   -33.33413265579       -2.84       -1.74    4.2    1.57s
  4   -33.33429105325       -3.80       -2.64    3.0    1.41s
  5   -33.33694082861       -2.58       -2.50    6.4    1.99s
  6   -33.33694156753       -6.13       -2.54    9.0    1.81s
  7   -33.33694383075       -5.65       -3.07    1.0    1.28s
  8   -33.33694385019       -7.71       -3.65    4.0    1.56s
  9   -33.33694392313       -7.14       -3.93    3.9    1.72s
 10   -33.33694392648       -8.48       -4.56    2.4    1.43s


When switching off explicitly the `LdosMixing`, by selecting `mixing=SimpleMixing()`,
the performance of number of required SCF steps starts to increase as we increase
the size of the modelled problem:

In [6]:
self_consistent_field(aluminium_setup(1); tol=1e-4, mixing=SimpleMixing());

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/XF0Hn/src/ASEconvert.jl:144
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -8.298295785008                   -0.85    5.4         
  2   -8.300242766211       -2.71       -1.59    1.0    117ms
  3   -8.300435171241       -3.72       -2.56    3.1    178ms
  4   -8.300389171270   +   -4.34       -2.31    5.2    236ms
  5   -8.300464105417       -4.13       -3.42    1.1    141ms
  6   -8.300464575311       -6.33       -3.80    2.1    201ms
  7   -8.300464639935       -7.19       -4.38    1.5    161ms


In [7]:
self_consistent_field(aluminium_setup(4); tol=1e-4, mixing=SimpleMixing());

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/XF0Hn/src/ASEconvert.jl:144
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -33.32396744576                   -0.56    7.9         
  2   -33.32527287462       -2.88       -1.27    1.8    1.12s
  3   -21.32801518692   +    1.08       -0.53    6.2    2.14s
  4   -33.24033848063        1.08       -1.52    5.0    1.99s
  5   -33.25830415863       -1.75       -1.36    2.9    1.48s
  6   -32.84578177494   +   -0.38       -1.22    3.6    1.56s
  7   -33.19074146621       -0.46       -1.46    4.0    1.58s
  8   -33.33539096085       -0.84       -2.13    2.6    1.30s
  9   -33.33487304639   +   -3.29       -2.32    2.9    1.49s
 10   -33.33625748425       -2.86       -2.57    1.9    1.22s
 11   -33.33675531885       -3.30       -2.83    2.5    1.37s
 12   -33.33694228836       -3.73      

For completion let us note that the more traditional `mixing=KerkerMixing()`
approach would also help in this particular setting to obtain a constant
number of SCF iterations for an increasing system size (try it!). In contrast
to `LdosMixing`, however, `KerkerMixing` is only suitable to model bulk metallic
system (like the case we are considering here). When modelling metallic surfaces
or mixtures of metals and insulators, `KerkerMixing` fails, while `LdosMixing`
still works well. See the Modelling a gallium arsenide surface example
or [^HL2021] for details. Due to the general applicability of `LdosMixing` this
method is the default mixing approach in DFTK.